In [16]:
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
import json
from datetime import datetime

In [22]:
def save_data_to_mongo(data_json, database_name,collection_name, mongo_url='mongodb://localhost:27017/', document_id='news'):
    """
    Save data to MongoDB collection, avoiding duplicates based on the 'url' field.

    :param data_json: List of dictionaries containing the data to be saved.
    :param mongo_uri: MongoDB URI.
    :param database_name: Name of the database.
    :param collection_name: Name of the collection.
    :param document_id: The document ID to update or insert data.
    """
    client_db = MongoClient(mongo_url)
    db = client_db[database_name]
    data_collection = db[collection_name]

    existing_document = data_collection.find_one({'_id': document_id})

    # Check for duplicates based on 'url' and append new data
    if existing_document and 'data' in existing_document:
        existing_data = existing_document['data']
        existing_urls = {item['url'] for item in existing_data}

        # Filter out new items that already exist based on the 'url' field
        new_data = [item for item in data_json if item['url'] not in existing_urls]

        if new_data:
            existing_data.extend(new_data)
            data_collection.update_one(
                {'_id': document_id},
                {'$set': {'data': existing_data}}
            )
    else:
        data_collection.update_one(
            {'_id': document_id},
            {'$set': {'data': data_json}},
            upsert=True
        )

    print("Data News Saved!")


In [24]:
MONGO_URI = 'mongodb://localhost:27017/'  # Update with your MongoDB URI
DATABASE_NAME = 'E2E'
DATA_COLLECTION_NAME = 'data_scrape'

In [25]:
# Create MongoDB client
client_db = MongoClient(MONGO_URI)
db = client_db[DATABASE_NAME]
data_collect = db[DATA_COLLECTION_NAME]

In [26]:
TOPIK  = 'instansi A'

In [27]:
path_folder = r'C:\Users\Win10\End-to-End-Testing-main\datacrawl'
datas = os.listdir(path_folder)
path_news = os.path.join(path_folder, datas[0])

with open(path_news, 'r') as f:
    news_dat = json.load(f)

news_df = pd.DataFrame(news_dat['articles']).head(3)

news_df['label_owner'] = TOPIK
news_df['label_timestamp'] = datetime.now().isoformat()
data_dict = news_df.to_dict("records")

In [28]:
data_dict

[{'judul': 'Profil Hetifah Sjaifudian, Anggota DPR RI Dapil Kalimantan Timur Periode 2024-2029 dan Rekam Jejak - Tribunkaltim.co',
  'ringkasan': 'Sebanyak 580 caleg berhasil memastikan diri lolos dan menjadi anggota DPR RI periode 2024-2029.\n\nDengan mendulang banyak suara, tentu Hetifah Sjaifudian bisa kembali mengamankan 1 kursi di Senayan.\n\nH...',
  'sumber': 'Tribun Kaltim',
  'tanggal': '2024-08-02T02:53:20Z',
  'url': 'https://kaltim.tribunnews.com/2024/08/02/profil-hetifah-sjaifudian-anggota-dpr-dapil-ri-kalimantan-timur-periode-2024-2029-dan-rekam-jejak',
  'label_owner': 'instansi A',
  'label_timestamp': '2024-08-06T12:00:18.982130'},
 {'judul': 'Menhub Budi Pede Tol Laut Lanjut di Masa Prabowo-Gibran, Ini Alasannya',
  'ringkasan': 'Budi meyakini bahwa program tol laut tidak hanya akan berjalan di pemerintah Presiden Joko Widodo (Jokowi), tetapi akan terus berlanjut di pemerintahan selanjutnya periode 2024-2029.\n\n"Saya pikir kare...',
  'sumber': 'VIVA.co.id',
  'tangg

In [29]:
save_data_to_mongo(data_dict, DATABASE_NAME, DATA_COLLECTION_NAME)

Data News Saved!


In [2]:
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
import json
from datetime import datetime

def save_data_to_mongo(data_json, database_name, collection_name, mongo_url='mongodb://localhost:27017/', document_id='news'):
    """
    Save data to MongoDB collection, avoiding duplicates based on the 'url' field.
    If 'url' exists but with a new 'label_owner', add the new 'label_owner' to the list.

    :param data_json: List of dictionaries containing the data to be saved.
    :param mongo_uri: MongoDB URI.
    :param database_name: Name of the database.
    :param collection_name: Name of the collection.
    :param document_id: The document ID to update or insert data.
    """
    client_db = MongoClient(mongo_url)
    db = client_db[database_name]
    data_collection = db[collection_name]

    existing_document = data_collection.find_one({'_id': document_id})

    if existing_document and 'data' in existing_document:
        existing_data = existing_document['data']
        existing_urls = {item['url'] for item in existing_data}

        for new_item in data_json:
            if new_item['url'] in existing_urls:
                for existing_item in existing_data:
                    if existing_item['url'] == new_item['url']:
                        if not isinstance(existing_item['label_owner'], list):
                            existing_item['label_owner'] = [existing_item['label_owner']]
                        if not isinstance(existing_item['label_timestamp'], list):
                            existing_item['label_timestamp'] = [existing_item['label_timestamp']]
                        
                        if new_item['label_owner'] not in existing_item['label_owner']:
                            existing_item['label_owner'].append(new_item['label_owner'])
                        existing_item['label_timestamp'].append(new_item['label_timestamp'])
                        break
            else:
                new_item['label_owner'] = [new_item['label_owner']]
                new_item['label_timestamp'] = [new_item['label_timestamp']]
                existing_data.append(new_item)

        data_collection.update_one(
            {'_id': document_id},
            {'$set': {'data': existing_data}}
        )
    else:
        for new_item in data_json:
            new_item['label_owner'] = [new_item['label_owner']]
            new_item['label_timestamp'] = [new_item['label_timestamp']]
        data_collection.update_one(
            {'_id': document_id},
            {'$set': {'data': data_json}},
            upsert=True
        )

    print("Data News Saved!")

MONGO_URI = 'mongodb://localhost:27017/'  # Update with your MongoDB URI
DATABASE_NAME = 'E2E'
DATA_COLLECTION_NAME = 'data_scrape'

# Create MongoDB client
client_db = MongoClient(MONGO_URI)
db = client_db[DATABASE_NAME]
data_collect = db[DATA_COLLECTION_NAME]



path_folder = r'C:\Users\Win10\End-to-End-Testing-main\datacrawl'
datas = os.listdir(path_folder)
path_news = os.path.join(path_folder, datas[0])

with open(path_news, 'r') as f:
    news_dat = json.load(f)

news_df = pd.DataFrame(news_dat['articles']).head(3)

news_df['label_owner'] = TOPIK
news_df['label_timestamp'] = datetime.now().isoformat()
data_dict = news_df.to_dict("records")

save_data_to_mongo(data_dict, DATABASE_NAME, DATA_COLLECTION_NAME)


Data News Saved!


In [4]:
news_df = pd.DataFrame(news_dat['articles']).tail(2)
TOPIK = 'instansi A'
news_df['label_owner'] = TOPIK
news_df['label_timestamp'] = datetime.now().isoformat()
data_dict = news_df.to_dict("records")

save_data_to_mongo(data_dict, DATABASE_NAME, DATA_COLLECTION_NAME)

Data News Saved!


In [5]:
data_dict

[{'judul': 'Jumlah Pekerja yang Kena PHK Capai 32 Ribu Orang, Jakarta Paling Banyak',
  'ringkasan': 'Massa buruh Konfederasi Serikat Pekerja Seluruh Indonesia (KSPSI) menggelar aksi demo di kawasan Patung Kuda, Monas, Jakarta, Rabu 3 Juli 2024. Dalam aksinya massa buruh menyerukan penolakan PHK pada ...',
  'sumber': 'TEMPO.CO',
  'tanggal': '2024-08-02T00:11:27Z',
  'url': 'https://bisnis.tempo.co/read/1898757/jumlah-pekerja-yang-kena-phk-capai-32-ribu-orang-jakarta-paling-banyak',
  'label_owner': 'instansi A',
  'label_timestamp': '2024-08-07T08:47:27.737476'},
 {'judul': 'Angela Tanoesoedibjo Gantikan HT Pimpin Partai Perindo, PKB: Banggakan Kaum Muda : Okezone Nasional',
  'ringkasan': 'JAKARTA - Ketua DPP PKB Daniel Johan turut mengucapkan selamat kepada Angela Tanoesoedibjo lantaran telah menjadi Ketua Umum Partai Perindo. Angela, kata dia, meneruskan tongkat estafet kepemimpinan p...',
  'sumber': 'https://nasional.okezone.com/',
  'tanggal': '2024-08-02T00:00:00Z',
  'url': '

# Load Data

In [6]:
def load_data_by_label_owner(database_name, collection_name, label_owner, mongo_url='mongodb://localhost:27017/', document_id='news'):
    """
    Load data from MongoDB collection based on label_owner.

    :param database_name: Name of the database.
    :param collection_name: Name of the collection.
    :param label_owner: The label owner to filter by.
    :param mongo_uri: MongoDB URI.
    :param document_id: The document ID to update or insert data.
    :return: Data filtered by label_owner.
    """
    client_db = MongoClient(mongo_url)
    db = client_db[database_name]
    data_collection = db[collection_name]

    existing_document = data_collection.find_one({'_id': document_id})
    
    if existing_document and 'data' in existing_document:
        filtered_data = [item for item in existing_document['data'] if label_owner in item['label_owner']]
        return filtered_data
    else:
        return []

In [7]:
# Load data by label_owner
label_owner_to_search = 'instansi B'
filtered_data = load_data_by_label_owner(DATABASE_NAME, DATA_COLLECTION_NAME, label_owner_to_search)
print(f"Data for label_owner '{label_owner_to_search}':")
print(filtered_data)

Data for label_owner 'instansi B':
[{'judul': 'Profil Hetifah Sjaifudian, Anggota DPR RI Dapil Kalimantan Timur Periode 2024-2029 dan Rekam Jejak - Tribunkaltim.co', 'ringkasan': 'Sebanyak 580 caleg berhasil memastikan diri lolos dan menjadi anggota DPR RI periode 2024-2029.\n\nDengan mendulang banyak suara, tentu Hetifah Sjaifudian bisa kembali mengamankan 1 kursi di Senayan.\n\nH...', 'sumber': 'Tribun Kaltim', 'tanggal': '2024-08-02T02:53:20Z', 'url': 'https://kaltim.tribunnews.com/2024/08/02/profil-hetifah-sjaifudian-anggota-dpr-dapil-ri-kalimantan-timur-periode-2024-2029-dan-rekam-jejak', 'label_owner': ['instansi A', 'instansi B'], 'label_timestamp': ['2024-08-06T12:00:18.982130', '2024-08-07T08:44:47.394531']}, {'judul': 'Menhub Budi Pede Tol Laut Lanjut di Masa Prabowo-Gibran, Ini Alasannya', 'ringkasan': 'Budi meyakini bahwa program tol laut tidak hanya akan berjalan di pemerintah Presiden Joko Widodo (Jokowi), tetapi akan terus berlanjut di pemerintahan selanjutnya periode 20

In [10]:
pd.DataFrame(filtered_data)

,judul,ringkasan,sumber,tanggal,url,label_owner,label_timestamp
0,"Profil Hetifah Sjaifudian, Anggota DPR RI Dapi...",Sebanyak 580 caleg berhasil memastikan diri lo...,Tribun Kaltim,2024-08-02T02:53:20Z,https://kaltim.tribunnews.com/2024/08/02/profi...,"[instansi A, instansi B]","[2024-08-06T12:00:18.982130, 2024-08-07T08:44:..."
1,Menhub Budi Pede Tol Laut Lanjut di Masa Prabo...,Budi meyakini bahwa program tol laut tidak han...,VIVA.co.id,2024-08-02T02:41:08Z,https://www.viva.co.id/bisnis/1738349-menhub-b...,"[instansi A, instansi B]","[2024-08-06T12:00:18.982130, 2024-08-07T08:44:..."
2,Sah! Pasangan Salim Fakhry-Heri Al Hilal Maju ...,"KUTACANE, Waspada.co.id - Dewan Pimpinan Pusat...",Waspada Online,2024-08-02T02:40:57Z,https://waspada.co.id/sah-pasangan-salim-fakhr...,"[instansi A, instansi B]","[2024-08-06T12:00:18.982130, 2024-08-07T08:44:..."
3,Menhub optimistis program tol laut berlanjut ...,Jakarta (ANTARA) - Menteri Perhubungan (Menhub...,Antara News,2024-08-02T00:19:01Z,https://ambon.antaranews.com/berita/223035/men...,[instansi B],[2024-08-07T08:46:33.374327]
4,"HARI INI, Aliansi Santri Gus Dur Menggugat Ge...","Laporan Wartawan Tribuncirebon.com, Handhika R...",Tribun cirebon,2024-08-02T00:16:22Z,https://cirebon.tribunnews.com/2024/08/02/hari...,[instansi B],[2024-08-07T08:46:33.374327]
5,Jumlah Pekerja yang Kena PHK Capai 32 Ribu Ora...,Massa buruh Konfederasi Serikat Pekerja Seluru...,TEMPO.CO,2024-08-02T00:11:27Z,https://bisnis.tempo.co/read/1898757/jumlah-pe...,"[instansi B, instansi A]","[2024-08-07T08:46:33.374327, 2024-08-07T08:47:..."
6,Angela Tanoesoedibjo Gantikan HT Pimpin Partai...,JAKARTA - Ketua DPP PKB Daniel Johan turut men...,https://nasional.okezone.com/,2024-08-02T00:00:00Z,https://nasional.okezone.com/read/2024/08/02/3...,"[instansi B, instansi A]","[2024-08-07T08:46:33.374327, 2024-08-07T08:47:..."
